In [14]:
import pandas as pd
import numpy as np
from random import random
from CTL2.causal_tree_learn import CausalTree
from DGP.dgp6 import dgp_ate4_train
from DGP.dgp6 import dgp_ate4_test
from MC.mc2 import MC_MSE_2
from MC.mc2_maxd import MC_MSE_2max
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse
import matplotlib.pyplot as plt
import pickle as pkl
#from notify_run import Notify 
#notify = Notify()
import datetime

In [35]:
a = np.zeros(10).reshape(-1,1)
b = np.append(b,a, axis =1)

In [40]:
b

array([[1., 1., 1., 0.],
       [1., 1., 1., 0.],
       [1., 1., 1., 0.],
       [1., 1., 1., 0.],
       [1., 1., 1., 0.],
       [1., 1., 1., 0.],
       [1., 1., 1., 0.],
       [1., 1., 1., 0.],
       [1., 1., 1., 0.],
       [1., 1., 1., 0.]])

In [41]:
np.var(b, axis = 1)

array([0.1875, 0.1875, 0.1875, 0.1875, 0.1875, 0.1875, 0.1875, 0.1875,
       0.1875, 0.1875])

In [53]:
def MC_ate4(n_train, n_test, var_e, reps, nomin_test):    
    #true ATE
    #ATE1 = np.ones(reps)*3
    #ATE2 = np.ones(reps)*1
    #ATE3 = np.ones(reps)*-2
    #ATE4 = np.ones(reps)*-3
    #predicted ate
    #ate1 = []
    #ate2 = []
    #ate3 = []
    #ate4 = []
    #TOTAL MSE, Bias, Var    
    tauhat = np.zeros((n_test, 1))
    x_test, y_test, treat_test = dgp_ate4_test(n_test, var_e)
    tau = np.where(((x_test[:,0] >= 0) & (x_test[:,1] >= 0)),3, np.where(((x_test[:,0] >= 0) & (x_test[:,1] < 0)),1, np.where(((x_test[:,0] < 0) & (x_test[:,1] >= 0)),-2,-3)))
    #MONTE CARLO
    for i in range(reps):
        x_train, y_train, treat_train = dgp_ate4_test(n_train,var_e)     
        #true individual treatment effect
        #tau = np.where(((x1 >= 0) & (x2 >= 0)),2, np.where(((x1 >= 0) & (x2 < 0)),1, np.where(((x1 < 0) & (x2 >= 0)),-1,-2)))
        #predicted infividual treatment effect
        ctl_predict = MC_MSE_2max(x_train, x_test, y_train, y_test, treat_train, treat_test, nomin_test)       
        tauhat = np.append(tauhat, ctl_predict.reshape(-1,1), axis = 1)

    #variance and mean across monte carlo simulations for each observation. 10000x1 arrays
    ind_var = np.var(tauhat[:, 1:], axis = 1)
    ind_mean = np.mean(tauhat, axis = 1)
    
    total_var = np.sum(ind_var)/n_test
    total_bias = np.sum(np.square(np.subtract(ind_mean, tau)))/n_test

    
    return total_var, total_bias, total_var+total_bias

In [47]:
MC_ate4(1000,10000, 0.01, 10, 0.5)

(10000, 11)
(10000, 10)
(10000,)
(10000,)


(0.21837717409120236, 0.16993763523699304, 0.3883148093281954)

In [6]:
def MC_ate8(n, var_e, reps, nomin_test):    
    #true ATE
    ATE1 = np.ones(reps)*5
    ATE2 = np.ones(reps)*-1
    ATE3 = np.ones(reps)*2.5
    ATE4 = np.ones(reps)*-3.5
    ATE5 = np.ones(reps)*3.5
    ATE6 = np.ones(reps)*-2.5
    ATE7 = np.ones(reps)*-1
    ATE8 = np.ones(reps)*-5
    #predicted ate
    ate1 = []
    ate2 = []
    ate3 = []
    ate4 = []
    ate5 = []
    ate6 = []
    ate7 = []
    ate8 = []
    #TOTAL MSE, Bias, Var
    MSE = []
    bias = []
    var = []
    n2 = int(n/2)
    
    leafs = []
    for i in range(reps):
        x_train, x_test, y_train, y_test, treat_train, treat_test = dgp_ate8(n,var_e)     
        #true individual treatment effect
        tau = np.where(((x_test[:,0] >= 0) & (x_test[:,1] >= 0) & (x_test[:,2] >= 0)),5, 
                       np.where(((x_test[:,0] < 0) & (x_test[:,1] >= 0) & (x_test[:,2] >= 0)), -1, 
                                np.where(((x_test[:,0] >= 0) & (x_test[:,1] < 0) & (x_test[:,2] >= 0)), 2.5, 
                                         np.where(((x_test[:,0] < 0) & (x_test[:,1] < 0) & (x_test[:,2] >= 0)), -3.5, 
                                                  np.where(((x_test[:,0] >= 0) & (x_test[:,1] >= 0) & (x_test[:,2] < 0)), 3.5, 
                                                           np.where(((x_test[:,0] < 0) & (x_test[:,1] >= 0) & (x_test[:,2] < 0)), -2.5, 
                                                                    np.where(((x_test[:,0] >= 0) & (x_test[:,1] < 0) & (x_test[:,2] < 0)), 1, -5)))))))
        
        
        #predicted infividual treatment effect
        ctl_predict = MC_MSE_2(x_train, x_test, y_train, y_test, treat_train, treat_test, nomin_test)
        #predicted ATE
        ate1 = np.append(ate1, np.mean(ctl_predict[np.where((x_test[:,0] >= 0) & (x_test[:,1] >= 0) & (x_test[:,2] >= 0))]))
        ate2 = np.append(ate2, np.mean(ctl_predict[np.where((x_test[:,0] < 0) & (x_test[:,1] >= 0) & (x_test[:,2] >= 0))]))
        ate3 = np.append(ate3, np.mean(ctl_predict[np.where((x_test[:,0] >= 0) & (x_test[:,1] < 0) & (x_test[:,2] >= 0))]))
        ate4 = np.append(ate4, np.mean(ctl_predict[np.where((x_test[:,0] < 0) & (x_test[:,1] < 0) & (x_test[:,2] >= 0))]))
        ate5 = np.append(ate5, np.mean(ctl_predict[np.where((x_test[:,0] >= 0) & (x_test[:,1] >= 0) & (x_test[:,2] < 0))]))
        ate6 = np.append(ate6, np.mean(ctl_predict[np.where((x_test[:,0] < 0) & (x_test[:,1] >= 0) & (x_test[:,2] < 0))]))
        ate7 = np.append(ate7, np.mean(ctl_predict[np.where((x_test[:,0] >= 0) & (x_test[:,1] < 0) & (x_test[:,2] < 0))]))
        ate8 = np.append(ate8, np.mean(ctl_predict[np.where((x_test[:,0] < 0) & (x_test[:,1] < 0) & (x_test[:,2] < 0))]))

        #leafs = np.append(leafs, len(np.unique(ctl_predict)))
        #nn = int(len(ctl_predict))
        MSE = np.append(MSE, mse(ctl_predict,tau))
        bias = np.append(bias, np.sum(np.subtract(tau,ctl_predict))/n2)
        var = np.append(var, np.var(np.subtract(tau,ctl_predict)))

    #ATE MSE's
    mean_ate = np.array([np.mean(ate1), np.mean(ate2), np.mean(ate3), np.mean(ate4), np.mean(ate5), np.mean(ate6), np.mean(ate7), np.mean(ate8)])    
    mse_ate = np.array([mse(ATE1, ate1), mse(ATE2, ate2),mse(ATE3, ate3), mse(ATE4, ate4),mse(ATE5, ate5), mse(ATE6, ate6), mse(ATE7, ate7), mse(ATE8, ate8)])
    bias_ate = np.array([np.sum(np.subtract(ATE1,ate1))/reps, np.sum(np.subtract(ATE2,ate2))/reps, np.sum(np.subtract(ATE3,ate3))/reps, np.sum(np.subtract(ATE4,ate4))/reps,
                         np.sum(np.subtract(ATE5,ate5))/reps, np.sum(np.subtract(ATE6,ate6))/reps, np.sum(np.subtract(ATE7,ate7))/reps, np.sum(np.subtract(ATE8,ate8))/reps])
    var_ate = np.array([np.var(ate1), np.var(ate2), np.var(ate3), np.var(ate4), np.var(ate5), np.var(ate6), np.var(ate7), np.var(ate8)])
    
    return mean_ate, mse_ate, bias_ate, var_ate, np.mean(MSE), np.mean(bias), np.mean(var)

In [57]:
print("start time: ", str(datetime.datetime.now())[11:16])
test_sizes = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]

MSE_ = []
bias_ = []
var_ = []

var_e = 0.01
n_test = 1000
n_train = 1000
reps = 500

for size in test_sizes:
    var, bias, MSE = MC_ate4(n_train, n_test, var_e, reps, size)
    MSE_ = np.append(MSE_, MSE)
    bias_ = np.append(bias_, bias)
    var_ = np.append(var_, var)
    
#notify.send('Done running the simulations')
print("end time: ", str(datetime.datetime.now())[11:16])

start time:  19:22
end time:  20:51


In [58]:
MSE_

array([0.54120302, 0.48323293, 0.53879839, 0.5371711 , 0.53870741,
       0.66777161, 0.83124283])

In [59]:
bias_

array([0.17176529, 0.18949092, 0.22313046, 0.22177298, 0.2390433 ,
       0.31482238, 0.44957316])

In [60]:
var_

array([0.36943772, 0.29374201, 0.31566793, 0.31539813, 0.29966411,
       0.35294922, 0.38166967])

#### True ATEs: 
* 5 
* -1
* 2.5
* -3.5
* 3.5 
* -2.5 
* 1 
* -5

In [17]:
mean_ate_.reshape(len(test_sizes),8).transpose()

array([[ 4.14855763,  4.02719516,  3.03399817],
       [-1.71754496, -1.79408631, -2.1092917 ],
       [ 2.21991572,  2.29791427,  2.2829809 ],
       [-3.66019621, -3.54938534, -2.81498489],
       [ 3.64552151,  3.54371049,  2.8274092 ],
       [-2.25032383, -2.36981421, -2.32688852],
       [ 1.70944201,  1.73778637,  2.06191387],
       [-4.1436783 , -4.02494506, -3.00283004]])

In [15]:
var_ate_.reshape(len(test_sizes),8).transpose()

array([[0.55601273, 0.34953758, 0.47392983],
       [0.64056395, 0.32628486, 0.47195182],
       [0.6607025 , 0.3365409 , 0.51385853],
       [0.59775113, 0.29948207, 0.48458515],
       [0.58298711, 0.36482551, 0.51193589],
       [0.61969623, 0.34605047, 0.50771609],
       [0.55941882, 0.26764132, 0.48111972],
       [0.60636176, 0.29157805, 0.43707566]])

In [61]:
df = pd.DataFrame([test_sizes, MSE_, bias_, var_]).transpose().set_index(0)
df.columns=['MSE_TOTAL', 'MSE_T_BIAS', 'MSE_T_VAR']

In [66]:
df

,MSE_TOTAL,MSE_T_BIAS,MSE_T_VAR
0,,,
0.2,0.541203,0.171765,0.369438
0.3,0.483233,0.189491,0.293742
0.4,0.538798,0.223130,0.315668
0.5,0.537171,0.221773,0.315398
0.6,0.538707,0.239043,0.299664
0.7,0.667772,0.314822,0.352949
0.8,0.831243,0.449573,0.381670


In [65]:
import pickle
with open('msetotal001.pkl','wb') as f:
    pickle.dump(df, f)

In [ ]:
mean_ate_ = mean_ate_.reshape(len(test_sizes),8).transpose()
mse_ate_ = mse_ate_.reshape(len(test_sizes),8).transpose()
bias_ate_ = bias_ate_.reshape(len(test_sizes),8).transpose()
var_ate_ = var_ate_.reshape(len(test_sizes),8).transpose()
df = pd.DataFrame([test_sizes, mean_ate_[0], mean_ate_[1],mean_ate_[2], mean_ate_[3], mse_ate_[0], mse_ate_[1],mse_ate_[2], mse_ate_[3], bias_ate_[0], bias_ate_[1], bias_ate_[2], bias_ate_[3], var_ate_[0], var_ate_[1],var_ate_[2], var_ate_[3], MSE_, bias_, var_]).transpose().set_index(0)
df.columns=['Mean1', 'Mean2', 'Mean3', 'Mean4', 'MSE1', 'MSE2','MSE3', 'MSE4', 'BIAS1', 'BIAS2','BIAS3', 'BIAS4', 'VAR1', 'VAR2','VAR3', 'VAR4', 'MSE_TOTAL', 'MSE_T_BIAS', 'MSE_T_VAR']